In [1]:
import spacy
from spacy import displacy

In [2]:
with open('case_example.txt', 'r') as file:
    # Read the contents of the file
    text = file.read()

In [14]:
import spacy_transformers

/Users/kowsalya/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [15]:
nlp_loaded = spacy.load('./model_ner')

/Users/kowsalya/anaconda3/lib/python3.11/site-packages/spacy_transformers/layers/hf_shim.py:137: UserWarning: Error loading saved torch state_dict with strict=True, likely due to differences between 'transformers' versions. Attempting to load with strict=False as a fallback...

If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current 'transformers' and 'spacy-transformers' versions. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [16]:
doc = nlp_loaded(text)

In [17]:
# Visualize the modified document with custom entities
displacy.serve(doc, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [ ]:
text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)

# Define your custom entities with start and end indexes
custom_entities = [(5, 21, "JUDGE"), (45, 51, "ORG")]

# Add the custom entities to the doc
for start, end, label in custom_entities:
    span = doc.char_span(start, end, label=label)
    if span is not None:
        doc.ents = list(doc.ents) + [span]

# Visualize the modified document with custom entities
displacy.serve(doc, style="ent")


In [ ]:
nlp_loaded = spacy.load('model_ner')
text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."


# Example: Process text using the loaded model
doc = nlp_loaded(text)

# Example: Print the entities in the processed text
for ent in doc.ents:
    print(ent.text, ent.label_)

In [ ]:
# Path where you want to save the model
output_dir = "model_ner"
nlp = spacy.load("en_legal_ner_trf")
# Save the model to the output directory
nlp.to_disk(output_dir)

In [ ]:
def jsonl_to_conll(jsonl_file, conll_file):
    with open(jsonl_file, 'r', encoding='utf-8') as f_jsonl, \
         open(conll_file, 'w', encoding='utf-8') as f_conll:
        
        for line in f_jsonl:
            data = json.loads(line)
            text = data['text']
            labels = data['label']
            
            token_start = 0
            token_end = 0
            token_label = 'O'
            
            for i, char in enumerate(text):
                if char.isspace() or i == len(text) - 1:
                    token_end = i if i != len(text) - 1 else i + 1
                    token_label = 'O'
                    for start, end, entity_type in labels:
                        if start <= token_start and end >= token_end:
                            if start == token_start:
                                token_label = 'B-' + entity_type
                            else:
                                token_label = 'I-' + entity_type
                    token_text = text[token_start:token_end]
                    f_conll.write(f"{token_text}\t{token_label}\n")
                    token_start = i + 1
            
            f_conll.write('\n')  # Separate sentences with a blank line

# Example usage
jsonl_file = 'admin.jsonl'
conll_file = 'admin.conll'
jsonl_to_conll(jsonl_file, conll_file)

In [ ]:
import json
import spacy
from spacy.tokens import DocBin, Span


In [20]:
def convert_jsonl_to_spacy(jsonl_file, spacy_output_file):
    nlp = spacy.blank("en")  # Load a blank English model
    doc_bin = DocBin()  # Initialize a DocBin to store the Spacy Docs

    with open(jsonl_file, "r", encoding="utf-8") as f:
        for line in f:
            data = json.loads(line.strip())
            text = data["text"]
            print(len(text))
            entities = data.get("label", [])

            doc = nlp.make_doc(text)  # Create a Spacy Doc object
            spans = []
            for start, end, label in entities:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    spans.append(span)
            doc.ents = spans  # Set the entities for the Doc
            doc_bin.add(doc)  # Add the Doc to the DocBin

    # Write the DocBin to file
    with open(spacy_output_file, "wb") as f:
        f.write(doc_bin.to_bytes())

# Example usage
jsonl_file = "admin.jsonl"
spacy_output_file = "output.spacy"
convert_jsonl_to_spacy(jsonl_file, spacy_output_file)

39638
47933
85838
55097
65272
11485
86548
59054
33217
52765


In [21]:
import json
import warnings

import spacy
from spacy.tokens import DocBin

def read_jsonl(fpath):
    with open(fpath, "r") as f:
        for line in f:
            yield json.loads(line)


nlp = spacy.blank("en")
doc_bin = DocBin()
docs = []
for data in read_jsonl("admin.jsonl"):
    doc = nlp.make_doc(data["text"])
    ents = []
    for entity in data["label"]:
        start = entity[0]
        end = entity[1]
        label = entity[2]
        span = doc.char_span(
            start_idx=start,
            end_idx=end,
            label=label,
            alignment_mode="strict",
        )
        if span is None:
            msg = (
                f"Skipping entity [{start}, {end}, {label}] in the "
                "following text because the character span "
                "'{doc.text[start:end]}' does not align with token "
                "boundaries:\n\n{repr(text)}\n"
            )
            warnings.warn(msg)
        else:
            ents.append(span)
    doc.set_ents(entities=ents)
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

/var/folders/p3/6d7myg7n3hd4c54fslcd_wzr0000gn/T/ipykernel_58599/3754166019.py:36: UserWarning: Skipping entity [19, 31, RESPONDENT] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:

{repr(text)}

  warnings.warn(msg)
/var/folders/p3/6d7myg7n3hd4c54fslcd_wzr0000gn/T/ipykernel_58599/3754166019.py:36: UserWarning: Skipping entity [36881, 36898, JUDGE] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:

{repr(text)}

  warnings.warn(msg)
/var/folders/p3/6d7myg7n3hd4c54fslcd_wzr0000gn/T/ipykernel_58599/3754166019.py:36: UserWarning: Skipping entity [4330, 4346, LAWYER] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:

{repr(text)}

  warnings.warn(msg)
/var/folders/p3/6d7myg7n3hd4c54fslcd_wzr0000gn/T/ipykernel_58599/3754166019.py:36: UserWarning: Skipping entity [4665, 4680, LAWYER] in the following text beca

/var/folders/p3/6d7myg7n3hd4c54fslcd_wzr0000gn/T/ipykernel_58599/3754166019.py:36: UserWarning: Skipping entity [19248, 19267, PRECEDENT] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:

{repr(text)}

  warnings.warn(msg)
/var/folders/p3/6d7myg7n3hd4c54fslcd_wzr0000gn/T/ipykernel_58599/3754166019.py:36: UserWarning: Skipping entity [41419, 41445, PRECEDENT] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:

{repr(text)}

  warnings.warn(msg)
/var/folders/p3/6d7myg7n3hd4c54fslcd_wzr0000gn/T/ipykernel_58599/3754166019.py:36: UserWarning: Skipping entity [41975, 41990, PRECEDENT] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:

{repr(text)}

  warnings.warn(msg)


In [22]:
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en")
db = DocBin().from_disk("train.spacy")
for doc in db.get_docs(nlp.vocab):
    assert doc.has_annotation("TAG")
    # or just inspect the tags
    for token in doc:
        print(token.text, token.tag_)

AssertionError: 